# Cas d'usage n°1

Après récupération d'un jeu de données quelconque (ici les industries classées pour la protection de l'environnement = ICPE de la région Hauts-de-France), l'objectif est de retrouver les codes communes manquants.

## Import des modules

In [1]:
import os
import pandas as pd
from requests_cache import CachedSession
from tqdm import tqdm

from french_cities import find_city

## Constitution du jeu de données : récupération des ICPE de la région Hauts-de-France

In [2]:
code_region = "32"
page_size = 1000

s = CachedSession()
r = s.get(
    "https://georisques.gouv.fr/api/v1/installations_classees",
    params={"page": "1", "page_size": page_size, "region": code_region},
)
number_pages = r.json()["total_pages"]
for x in tqdm(range(number_pages), desc="querying georisques"):
    try:
        data
    except NameError:
        data = []
    else:
        r = s.get(
            "https://georisques.gouv.fr/api/v1/installations_classees",
            params={
                "page": x + 1,
                "page_size": page_size,
                "region": code_region,
            },
        )
    finally:
        data += r.json()["data"]
    if not r.json()["next"]:
        break
data = pd.DataFrame(data)

querying georisques:  92%|██████████████████████████████████████████████████████████████▊     | 12/13 [00:00<00:00, 20.52it/s]


## Analyse du jeu de données obtenu

In [3]:
print(data.shape)
data.head()

(12136, 31)


,raisonSociale,adresse1,codePostal,codeInsee,commune,longitude,latitude,bovins,porcs,volailles,...,serviceAIOT,regime,rubriques,inspections,documentsHorsInspection,date_maj,adresse2,codeNaf,siret,adresse3
0,ISDI LABALETTE Denis,Lieu-dit Le Dièvre,62860,62739,Sains-lès-Marquion,3.083774,50.176359,False,False,False,...,DREAL HdF,Enregistrement,"[{'numeroRubrique': '2760', 'nature': 'Install...",[{'dateInspection': '2017-09-11'}],[{'identifiantFichier': 'ZRFGgSoRBn1aqafaBiI4a...,2024-02-21/15-57-04,NaN,NaN,NaN,NaN
1,ISDI de Saint-Laurent-Blangy,Zone des Trois Fontaines,62223,62753,Saint-Laurent-Blangy,2.809392,50.303387,False,False,False,...,DREAL HdF,Non ICPE,[],[{'dateInspection': '2019-02-04'}],[],2024-05-30/10-47-45,rue Henri Becquerel,NaN,NaN,NaN
2,RECYCL'ELECTRONIC SARL,2 Bis Rue <charles Duquesnoy,62270,62361,Frévent,2.293696,50.267633,False,False,False,...,DREAL HdF,Autres régimes,[],[{'dateInspection': '2015-09-29'}],[],2024-05-15/17-11-35,NaN,38,53748769600016,NaN
3,ISDI -TCPA (PATINIER A),rue de la Gare,62470,62197,Camblain-Châtelain,2.429339,50.478130,False,False,False,...,DREAL HdF,Non ICPE,[],[{'dateInspection': '2016-12-06'}],[],2024-05-30/10-47-45,Hâmeau de La Ferté au lieu dit Le Petit Pingue...,NaN,33255491400025,NaN
4,LAV'ALIM,Zone Industrielle Le Royeux,02430,02340,Gauchy,3.293073,49.822421,False,False,False,...,DREAL HdF,Autres régimes,[],[{'dateInspection': '2020-02-24'}],[],2023-11-29/11-19-15,5 Avenue de l'Europe,81,48908926800036,NaN


## ICPE dépourvues de codes commune INSEE

In [4]:
data.codeInsee.isnull().value_counts().to_frame("Nombre de codes manquants :")

,Nombre de codes manquants :
codeInsee,
False,11881
True,255


##### A date du 30/05/2024, le jeu de données est constitué de 12117 lignes et 31 colonnes, dont 255 codes communes INSEE manquants. Pourtant, d'autres champs
sont disponibles et exploitables pour retrouver les communes manquantes :
* des champs adresse ;
* un libellé de commune ;
* des coordonnées géographiques et référentiel de projection ;
* un code postal.

## Complétion des données avec `french-cities`
### Configuration de l'API INSEE

In [5]:
os.environ["insee_key"] = "********************"
os.environ["insee_secret"] = "********************"

### Extraction des données manquantes

In [6]:
missing = data[data.codeInsee.isnull()].copy()

Au besoin, vérifier que le système de projection des coordonnées est en EPSG 2154 (lorsqu'il est connu) :

In [7]:
missing[["systemeCoordonneesAIOT"]].drop_duplicates()

,systemeCoordonneesAIOT
17,2154
499,NaN


### Concaténation des adresses :

In [8]:
cols = [f"adresse{x}" for x in range(1, 4)]
addresses = (
    missing[cols[0]]
    .str.cat(missing[cols[1:]], sep=" ", na_rep="")
    .str.replace(" +", " ", regex=True)
    .str.strip(" ")
)
missing["adresse"] = addresses

### Recherche des communes manquantes à l'aide de `french-cities` :

In [9]:
filled = find_city(
    missing,
    year="last",
    x="coordonneeXAIOT",
    y="coordonneeYAIOT",
    epsg=2154,
    city="commune",
    dep=False,
    address="adresse",
    postcode="codePostal",
    use_nominatim_backend=False,
    field_output="newCodeInsee",
)
filled

,raisonSociale,adresse1,codePostal,codeInsee,commune,longitude,latitude,bovins,porcs,volailles,...,rubriques,inspections,documentsHorsInspection,date_maj,adresse2,codeNaf,siret,adresse3,adresse,newCodeInsee
index,,,,,,,,,,,,,,,,,,,,,
17,Mairie de FAMPOUX,Lieu-dit Les Huit Cannes,62118,NaN,Fampoux,-2.279526,45.502618,False,False,False,...,"[{'numeroRubrique': '2760', 'nature': 'Install...",[{'dateInspection': '2020-02-20'}],[{'identifiantFichier': 'xwOI57mkibOLwaTaqUaWl...,2023-11-29/11-19-15,NaN,84,21620323200014,NaN,Lieu-dit Les Huit Cannes,62323
499,GSMC Market,ZAC DU PONT SANS PAREIL,62610,NaN,BOIS EN ARDRES,NaN,NaN,False,False,False,...,[],[{'dateInspection': '2017-11-10'}],[],2024-06-12/12-42-23,NaN,NaN,75202140200025,NaN,ZAC DU PONT SANS PAREIL,62038
1424,THESEE FORMATION,Domaine des Vivirets,60490,NaN,MARQUEGLISE,NaN,NaN,False,False,False,...,[],"[{'dateInspection': '2022-03-22', 'fichierInsp...",[],NaN,NaN,NaN,NaN,NaN,Domaine des Vivirets,60386
3613,ALTEA MSO,ZAC du Plateau,02200,NaN,Ploisy,NaN,NaN,False,False,False,...,[],[],[{'identifiantFichier': '10a4a1f0d48c4e9792982...,1900-04-25/00-09-21,NaN,NaN,74572170400039,NaN,ZAC du Plateau,02607
3689,LES FRUITS ROUGES AISNE SA,1 rue Jean Bodin,02000,NaN,Laon,-1.363081,-5.983856,False,False,False,...,"[{'numeroRubrique': '1511', 'nature': 'Entrepô...","[{'dateInspection': '2023-06-12', 'fichierInsp...",[{'identifiantFichier': '8abb00ab7c503f14017c5...,2024-02-22/12-08-16,NaN,46,37897020600025,NaN,1 rue Jean Bodin,02408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12041,CRAPET Jean-Louis,le fond du Marquoi,62550,NaN,Bailleul-lès-Pernes,-5.525510,42.559454,False,False,False,...,[],"[{'dateInspection': '2023-10-06', 'fichierInsp...",[],2023-11-11/02-33-09,Chemin de la Drève,NaN,88407344600012,NaN,le fond du Marquoi Chemin de la Drève,62071
12058,PRESSING DES PORTES DE L'AVESNOIS,RTE DE VALENCIENNES,59530,NaN,LE QUESNOY,-0.001143,0.001046,False,False,False,...,[],"[{'dateInspection': '2023-10-11', 'fichierInsp...",[],2024-02-08/02-33-46,CC LES PORTES DE L'AVESNOIS ZAE OUEST,96,85053052800015,NaN,RTE DE VALENCIENNES CC LES PORTES DE L'AVESNOI...,59481
12059,VAL PRESS,72 AV DE DENAIN,59300,NaN,VALENCIENNES,-0.001105,0.001474,False,False,False,...,[],"[{'dateInspection': '2023-10-11', 'fichierInsp...",[],2023-12-08/02-36-48,NaN,96,43404025900013,NaN,72 AV DE DENAIN,59606


### Réinjection les codes manquants dans le dataframe complet :

In [10]:
data = data.join(filled[["newCodeInsee"]])
data["codeInsee"] = data["codeInsee"].combine_first(data["newCodeInsee"])
data = data.drop("newCodeInsee", axis=1)

### Evaluation des données

In [11]:
data.codeInsee.isnull().value_counts().to_frame("Nombre de codes manquants :")

,Nombre de codes manquants :
codeInsee,
False,12135
True,1


### Données toujours manquantes

In [12]:
data[data.codeInsee.isnull()]

,raisonSociale,adresse1,codePostal,codeInsee,commune,longitude,latitude,bovins,porcs,volailles,...,serviceAIOT,regime,rubriques,inspections,documentsHorsInspection,date_maj,adresse2,codeNaf,siret,adresse3
8425,ATELIERS DES EPICES ET CONDIMENTS,19 RUE DE LA GARE,62360,NaN,PONT DE BRIQUES,NaN,NaN,False,False,False,...,DREAL HdF,Non ICPE,[],[{'dateInspection': '2015-01-12'}],[],2024-05-30/10-47-45,SAINT ETIENNE AU MONT,NaN,53197729600023,NaN


A date du 30/05/2024, une seule commune n'a pas été trouvée.

Effectivement, dans ce cas de figure, le lieu-dit (PONT DE BRIQUES) et la commune (SAINT ETIENNE AU MONT) ont été inversés : ceci explique que le score de la base adresse nationale n'ait pas été jugé suffisamment bon pour que le résultat de Saint-Etienne-au-Mont puisse être retenu...

Si cette fois, on décide d'utiliser l'API Nominatim en dernier recours, le code devient :

In [13]:
# On isole la(es) ligne(s) manquante(s)
missing = data[data.codeInsee.isnull()].copy()

# On concatène de nouveau les champs adresses :
cols = [f"adresse{x}" for x in range(1, 4)]
addresses = (
    missing[cols[0]]
    .str.cat(missing[cols[1:]], sep=" ", na_rep="")
    .str.replace(" +", " ", regex=True)
    .str.strip(" ")
)
missing.loc[:, "adresse"] = addresses

# Et on spécifie l'usage de Nominatim
missing = find_city(
    missing,
    year="last",
    x=False,
    y=False,
    dep=False,
    epsg=2154,
    city="commune",
    address="adresse",
    postcode="codePostal",
    use_nominatim_backend=True,
    field_output="newCodeInsee",
)

# Présentation des résultats
missing["newCodeInsee"]

index
8425    62746
Name: newCodeInsee, dtype: object

L'exécution de Nominatim ne conduit pas systématiquement au même résultat (et même parfois ne produit pas de résultat). Cela n'est pas totalement absurde, le hameau manquant étant à cheval sur plusieurs communes. Les résultats fournis restent  généralement pertinents.